In [19]:
import pyodbc
import re
import pandas as pd

In [20]:
# Buat koneksi ke MS Access Database
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\andikamcenroe\Documents\Knowledge\Analisa_Request\Database1.accdb;')
# Query Datanya dan tampung ke Dataframe
Inbox = pd.read_sql('SELECT * FROM Inbox', conn)
# Close connection
conn.close()


In [21]:
# Checkpoint Max / Min Date in dataframe
Inbox['Created'].describe(datetime_is_numeric=True)

count                             8813
mean     2021-01-30 11:16:00.144445696
min                2020-10-23 01:04:34
25%                2020-12-15 13:38:51
50%                2021-02-02 10:52:49
75%                2021-03-16 09:44:16
max                2021-04-23 16:47:12
Name: Created, dtype: object

In [22]:
# Filter dates & PIC
Inbox_Filt = Inbox[Inbox['Created'] >= '2021-04-01 00:00:00']
Inbox_Filt = Inbox_Filt[Inbox_Filt['From'].isin(["Mohamad SEPTIAN FAUZI (400074)",
                                      "Ria Astriana SURDIN",
                                      "Henri Bin Husni",
                                      "Saiful Anwar",
                                      "Andika Mcenroe"])]

In [23]:
# Fix Text Format (Remove line Feed, tab etc..)
Inbox_Filt['Contents'] = Inbox_Filt['Contents'].str.replace('[\r\n\t]','',regex=True)

In [24]:
# Tipe email adalah Email Balasan, cirinya ada Prefix RE: dan mengandung Attachment
Inbox_Filt = Inbox_Filt[(Inbox_Filt['Subject Prefix'] == "RE: ") & (Inbox_Filt['Has Attachments'] == True)] 

In [25]:
# Retrieve Last Contains Emails (Last replied Word by requester)
Inbox_Filt['Last_Contents'] = Inbox_Filt['Contents'].str.extract('(.+?(?=From: ))')

In [26]:
# Cek Kata2 Please Find dari Body Email terakhir
Inbox_Filt = Inbox_Filt[(Inbox_Filt['Last_Contents'].str.contains('please find',flags=re.IGNORECASE)) | 
                       (Inbox_Filt['Last_Contents'].str.contains('pleas find',flags=re.IGNORECASE))] 

In [27]:
# Ambil Bagian From pada keselururan Body Email untuk Mencari kepada siapa Tim MIS Reply Email
Inbox_Filt['Reply_From'] = Inbox_Filt['Contents'].str.extract('(?<=From: )(.*?)(?= Sent:)')

In [28]:
# Hilangkan <1bankid@dbs.com> JIKA ADA
Inbox_Filt['Reply_From'] = Inbox_Filt['Reply_From'].str.replace(' <(.*?)>','',regex=True)

In [29]:
# Mencari kapan Email ini dikirim sama requested
Inbox_Filt['DTTime_Req'] = Inbox_Filt['Contents'].str.extract('(?<=Sent: )(.*?)(?=To:)')

In [30]:
# Mencari kapan Email ini dikirim sama requested
Inbox_Filt['Tanggal'] = pd.to_datetime(Inbox_Filt['DTTime_Req'],format='%A, %B %d, %Y %I:%M %p')

In [31]:
# Filter yang Bukan Request yang Reply dari PIC sendiri
Inbox_Filt = Inbox_Filt[Inbox_Filt['From'] != Inbox_Filt['Reply_From']]

In [32]:
# Tulis ke Excel
Inbox_Filt.to_excel(r'C:\Users\andikamcenroe\Documents\Knowledge\Analisa_Request\Handle_Adhock_Baru_py.xlsx',index=False) 

## End Of Program

In [ ]:
Inbox_Filt.iloc[:,0:18].to_excel(r'C:\Users\andikamcenroe\Documents\Knowledge\Analisa_Request\Handle_Adhock_Baru_py_I.xlsx',index=False) 

In [73]:
Inbox_Filt.columns

Index(['ID', 'Importance', 'Icon', 'Priority', 'Subject', 'From',
       'Message To Me', 'Message CC to Me', 'Sender Name', 'CC', 'To',
       'Received', 'Message Size', 'Contents', 'Created', 'Modified',
       'Subject Prefix', 'Has Attachments', 'Normalized Subject',
       'Object Type', 'Content Unread', 'Last_Contents'],
      dtype='object')